In [ ]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GUTENBERG_CONFIG, GUTENBERG_DPO_CONFIG, SMOL_LM_135M

In [ ]:
cfg = GUTENBERG_CONFIG
cfg.model_id_or_path = SMOL_LM_135M
cfg.eval_steps = 100
cfg.max_samples = 100
cfg.eval_data_mode = "fixed"
wrapper = TrainerWrapper(cfg, True)

In [ ]:
wrapper.init_model()

In [ ]:
wrapper.init_data_module()

In [ ]:
wrapper.init_trainer()

In [ ]:
wrapper.trainer.evaluate()

In [ ]:
wrapper.train()